Набор функций  $[f_{1}, ... ,f_{n}] $ линейно зависим <=> $ \sum \limits_{i=1}^{n} \alpha_{i}f_{i} = 0, \alpha_{i} \not= 0, \forall x \in \mathbb{R} $,

Иначе $ f_{1} = - {\sum \limits_{i=2}^{n} \beta_{i} f_{i}}, \beta_{i} = {\alpha_{i} \over \alpha_{1}} $ ,

Можно составить линейную систему уравнений:,

$$ \overrightarrow{c} = A * \overrightarrow{x}, c_{i} = f_{1}(x_{i}), x_{i} = \beta_{i} $$,

где решение системы есть набор коэффициентов показывающий линейную зависимость.
Если решения нету или один из коэффициентов равен 0 => набор функций линейно независим

__1.__ Исследовать на линейную зависимость:

$$f_{1}(x)=e^{x}, f_{2}(x)=1, f_{3}(x)=x+1, f_{4}(x)=x-e^{x}.$$


In [394]:
import numpy as np
from collections import Counter
import json

def lindep(functions_list: list, start_x=0):
    dim = len(functions_list)
    used_dim = dim - 1
    
    c = np.asarray([functions_list[0](x) for x in range(start_x, start_x + used_dim)])
    data_array = np.asarray([[-function(x) for function in functions_list[1:]] for x in range(start_x, start_x + used_dim)])
    try:
        x = np.linalg.solve(data_array, c)
        x = np.insert(x, 0, 1)
    except np.linalg.LinAlgError:
        x = None
    return x

def is_lindep(functions_list, start_x = 0, count=2, step=10):
    cur_x = start_x
    result = []
    lindep_res = True
    result_solution = np.zeros(len(functions_list))
    prev_solution = None
    while count != 0:
        solution = lindep(functions_list=functions_list, start_x=cur_x)
        if prev_solution is None:
            prev_solution = solution
        else:
            if abs((solution - prev_solution).max()) < 0.05:
                solution = prev_solution
            else:
                prev_solution = solution
        result.append(json.dumps(list(solution)))
        cur_x += step
        count -= 1
    counter = Counter(result)
    if len(counter.keys()) == 1:
        for key in counter.keys():
            result_solution = np.asarray(json.loads(key))
            if np.all(result_solution==0):
                lindep_res = False
            
    else:
        lindep_res = False
        
    return lindep_res, result_solution, counter

In [395]:
def f1(x):
    return np.exp(x)

def f2(x):
    return 1

def f3(x):
    return x+1

def f4(x):
    return x - np.exp(x)

In [396]:
functions_list = [f1, f2, f3, f4]
lindep_res, solution, counter = is_lindep(functions_list=functions_list, start_x=0)

In [397]:
lindep_res, solution

(True, array([ 1.,  1., -1.,  1.]))

Решение одинаково для разных x => линейно зависимы

Проверка:

$ 1 * f1 + 1 * f2 + (-1) * f3 + 1 * f4 = e^{x} + 1 - (x + 1) + x - e^{x} = 0 , \forall x \in \mathbb{R}$

__2.__ Исследовать на линейную зависимость:
$$f_{1}(x)=2, f_{2}(x)=x, f_{3}(x)=x^{2}, f_{4}(x)=(x+1)^{2}$$

In [398]:
def f1(x):
    return 2

def f2(x):
    return x

def f3(x):
    return x ** 2

def f4(x):
    return (x+1) ** 2

In [399]:
functions_list = [f1, f2, f3, f4]
lindep_res, solution, counter = is_lindep(functions_list=functions_list, start_x=10, step=10)

In [401]:
lindep_res, solution

(True, array([ 1.,  4.,  2., -2.]))

Решения одинаковы для разных x

Проверка:

$ 1 * f1 + 4 * f2 + 2 * f3 - 2 * f4 = 2 + 4 * x + 2 * x^{2} - 2 * (x+1)^{2} = 2 + 4x + 2x^{2} - 2x^{2} - 4x - 2 = 0, \forall x \in \mathbb{R}$

Любое преобразование системы координат можно представить в вииде сложения трех независимых операций: сдвига, поворота и масштабирования.

Масштабирование определяется как $ \overrightarrow{x} = k*\overrightarrow{x_{0}} $

Вектор сдвига определяется $ \overrightarrow{l} = \overrightarrow{x} - \overrightarrow{x_{0}} $. И зависит только от выбора исходной точки отсчета. По умолчанию можно считать его нулем.

А поворот, математически можно представить в виде матрицы поворота $ R_{\phi}$.

Таким образом $ \overrightarrow{x} = R_{\phi} * \overrightarrow{x_{0}} $

В конечном итоге $ \overrightarrow{x} = \overrightarrow{l} +k * R_{\phi} * \overrightarrow{x_{0}} $

В многомерном пространстве матрица поворота может быть определена через независимые повороты вокруг базисных векторов => 

$ R_{...\alpha_{i}...} = R_{1}(\alpha_{1})*R_{2}(\alpha_{2})*R_{3}(\alpha_{3})*...*R_{n}(\alpha_{n}) , \forall x \in \mathbb{R}^n $

Аналогичные операции можно проводить и с функцианальным базисом { $ f1(x), ..., fn(x) $ } $ \in \mathbb{R}^3[x] $

In [420]:
class CoordSystem:
    def __init__(self, dim = 2, base: list=None):
        self.dim = dim
        if base is None:
            self.base = [[(lambda x: 0) if i != d else (lambda x: 1) for i in range(dim)] for d in range(dim)]
        else:
            self.base = self.__get_func_base(base)
        self.ch_base = self.base
        self.R = np.eye(self.dim)
        self.l = np.zeros((self.dim,))
        self.k = np.ones(self.dim)
        self.start_x = 0
            
    def change_base(self, new_base: list = None, zero_point: list=0, start_x = 0):
        self.start_x = start_x
        if zero_point != 0:
            self.l = np.asarray(zero_point)
        else:
            self.l = np.zeros(self.dim)
        self.k = np.ones(self.dim)
        func_base = self.__get_func_base(new_base)
        self.ch_base = func_base
        self.__find_rotate(func_base)
        self.__find_k(func_base)
    
    def get_ch_coord(self, x):
        x_kr = self.__get_rotate_coord(x)
        x_new = x_kr + self.l
        return x_new
    
    def func_reverse(self, x):
        x_r = []
        for i, x_i in enumerate(x):
            func_base_i = self.ch_base[i][i]
            i_data = func_base_i(self.start_x) / self.base[i][i](self.start_x)
            x_r.append(x_i*i_data)
        return np.asarray(x_r)
    
    def __get_rotate_coord(self, x):
        x = np.asarray(x)
        x_r = np.dot(self.R, x)
        x_kr = self.k * x_r
        
        return x_kr
    
    def __get_func_base(self, base_list:list = None):
        func_base = []
        if len(base_list) != self.dim:
            print("not equal dimentions")
            raise np.linalg.LinAlgError
        else:
            for i, n_b in enumerate(base_list):
                if isinstance(n_b, int) or isinstance(n_b, float):
                    func_base.append([(lambda x: 0) if d != i else (lambda x, current_value=n_b: current_value) for d in range(self.dim)])
                elif isinstance(n_b, list):
                    f_b = []
                    for j, b in enumerate(n_b):
                        if isinstance(b, int) or isinstance(b, float):
                            f_b.append(lambda x, current_value=b: current_value)
                        else:
                            f_b.append(b)
                    func_base.append(f_b)
                else:
                    func_base.append([(lambda x: 0) if d != i else n_b for d in range(self.dim)])
        
        return func_base
    
    def __find_k(self, func_base):
        new_base_result = self.__get_base_result(func_base)
        base_result = self.__get_base_result(self.base)
        
        for i in range(self.dim):
            self.k[i] = base_result[i][i] / (self.__get_rotate_coord(new_base_result[i])[i])
        
    
    def __get_base_data(self, base, x):
        result = [[b[i](x) for i in range(self.dim)] for b in base]
        return result
    
    def __get_base_result(self, base):
        # for all x
        x = self.start_x
        count = 100
        while count != 0:
            try:
                new_base_result = self.__get_base_data(base, x)
                break
            except Exception:
                x += 1
                count -= 1
        if count == 0:
            print("can not find functional solution")
            raise np.linalg.LinAlgError
            
        return new_base_result
    
    def __find_2d_rotation(self, new_data, data):
        A = np.asarray([[data[0], -data[1]],[data[1], data[0]]])
        y = np.asarray(new_data)
        x = np.linalg.solve(A, y)
        return x, 1
    
    def __get_2d_R_matrix(self, r_data):
        return np.asarray([[r_data[0], -r_data[1]],[r_data[1], r_data[0]]])
    
    def __get_nd_R_matrix(self, r_data, x1, x2):
        rotation_nd = np.eye(self.dim)
        rotation_nd[x1,x1] = r_data[0]
        rotation_nd[x2,x2] = r_data[0]
        rotation_nd[x1,x2] = -r_data[1]
        rotation_nd[x2,x1] = r_data[1]
        
        return rotation_nd
        
    def __find_rotate(self, new_base):
            new_base_result = self.__get_base_result(new_base)
            base_result = self.__get_base_result(self.base)
            print(base_result, new_base_result)
                
            if self.dim > 2:
                rotations = []
                for i in range(self.dim):
                    for j in range(i+1, self.dim):
                        i_data = [0,base_result[i][i]]
                        i_base = [new_base_result[j][k] for k in range(self.dim) if k == i or k == j]
                        r_data, k = self.__find_2d_rotation(i_data, i_base)
                        rotation_i = self.__get_nd_R_matrix(r_data*k, i, j)
                        rotations.append(rotation_i)
                self.R = np.linalg.multi_dot(rotations)
                    
            else:
                i_data = base_result[0]
                i_base = new_base_result[1]
                r_data, self.k = self.__find_2d_rotation(i_data, i_base)
                self.R = self.__get_2d_R_matrix(r_data)
        
        

__3.__ Найти координаты вектора $x = (2, 3, 5)\in \mathbb{R}^{3}$ в базисе $b_{1}=(0, 0, 10)$, $b_{2}=(2, 0, 0)$, $b_{3}=(0, 1, 0)$.

In [421]:
x = np.asarray([2,3,5])
e = np.asarray([1,1,1])
b = np.asarray([2,1,10])

In [422]:
k = b / e

In [423]:
x_b = x / k

In [424]:
x_b

array([1. , 3. , 0.5])

In [425]:
dim = 3
system = CoordSystem(dim = dim)

In [426]:
base = (2, 1, 10)
system.change_base(base)
system.l, system.R, system.k

[[1, 0, 0], [0, 1, 0], [0, 0, 1]] [[2, 0, 0], [0, 1, 0], [0, 0, 10]]


(array([0., 0., 0.]),
 array([[0.1 , 0.  , 0.  ],
        [0.  , 0.1 , 0.  ],
        [0.  , 0.  , 0.01]]),
 array([ 5., 10., 10.]))

In [427]:
x = (2,3,5)
system.get_ch_coord(x)

array([1. , 3. , 0.5])

__4.__ Найти координаты вектора $3x^{2}-2x+2\in\mathbb{R}^{3}[x]$:

а) в базисе $1$, $x$, $x^{2}$;

б) в базисе $x^{2}$, $x-1$, $1$.

$ 3x^2 - 2x + 2*1 = 3x^2 -2(x-1) + 0*1 $ => a) coord = (3, -2, 2), b) coord = (3, -2, 0) 

введем базис для функции { $ x^2, x, 1 $ }

In [467]:
base = [lambda x: x**2, lambda x: x, lambda x: 1]
base_2 = [lambda x: x**2, lambda x: x-1, lambda x: 1]

Проверка на линейную зависимость

In [468]:
lindep_res_base, solution_base, counter_base = is_lindep(base, start_x=10, step=1)
lindep_res_base_2, solution_base_2, counter_base_2 = is_lindep(base_2, start_x=10, step=1)

In [469]:
lindep_res_base, lindep_res_base_2

(False, False)

Разные решения для разных x => линейно независимы

In [470]:
system = CoordSystem(dim=3, base=base)

In [471]:
search_x = (3, -2, 2)
system.get_ch_coord(search_x)

array([ 3., -2.,  2.])

До смены системы координат вектор такой же

In [472]:
system.change_base(new_base=base_2, zero_point=(0,0,-2), start_x=1000)
system.R, system.l, system.k

[[1000000, 0, 0], [0, 1000, 0], [0, 0, 1]] [[1000000, 0, 0], [0, 999, 0], [0, 0, 1]]


(array([[1.001001e+09, 0.000000e+00, 0.000000e+00],
        [0.000000e+00, 1.001001e+06, 0.000000e+00],
        [0.000000e+00, 0.000000e+00, 1.000000e+09]]),
 array([ 0,  0, -2]),
 array([9.99e-10, 1.00e-06, 1.00e-09]))

In [473]:
system.get_ch_coord(x=search_x)

array([ 3.      , -2.002002,  0.      ])

__1.__ Найти скалярное произведение векторов $x, y \in \mathbb{R}$:<br>
а) $x=(0,-3, 6),~y=(-4, 7, 9);$<br>
б) $x=(7, -4, 0, 1),~y=(-3, 1, 11, 2).$


In [477]:
def get_scalar(x, y):
    return (np.asarray(x) * np.asarray(y)).sum()

In [478]:
x = (0,-3,6)
y = (-4,7,9)
get_scalar(x,y)

33

In [479]:
x = (7,-4,0,1)
y = (-3,1,11,2)
get_scalar(x,y)

-23

__2.__ Найти нормы векторов $(4, 2, 4)$ и $(12, 3, 4)$ и угол между ними.


In [483]:
def get_vector_norm(x):
    return np.linalg.norm(x)

def get_vector_angle(x, y):
    x_n = get_vector_norm(x)
    y_n = get_vector_norm(y)
    scalar = get_scalar(x,y)
    angle = np.arccos(scalar / (x_n * y_n)) * 180 / np.pi
    
    return angle

In [486]:
x = (4,2,4)
y = (12,3,4)
get_vector_norm(x), get_vector_norm(y), get_vector_angle(x,y)

(6.0, 13.0, 26.176952171666542)

__3.__ Будет ли линейное пространство евклидовым, если за скалярное произведение принять:<br>
а) произведение длин векторов;<br>
б) утроенное обычное скалярное произведение векторов?<br>

In [520]:
def check(scalar):
    x = np.asarray([1,2,3])
    y = np.asarray([2,8,4])
    x_1 = np.asarray([3,10,8])
    return cond_1(x,y,scalar) and cond_2(x,y,scalar) and cond_3(x, x_1, y, scalar) and cond_4(x, scalar)

def cond_1(x, y, scalar):
    return scalar(x,y) == scalar(y,x)

def cond_2(x,y,scalar):
    return scalar(3*x, y) == 3*scalar(x,y)

def cond_3(x1, x2, y, scalar):
    return scalar(x1 + x2, y) == scalar(x1,y) + scalar(x2,y)

def cond_4(x, scalar):
    return (scalar(x,x) >= 0) and (scalar(0,0) == 0)

In [521]:
check(scalar=lambda x, y: get_vector_norm(x) * get_vector_norm(y))

False

In [522]:
check(scalar=lambda x, y: 3 * get_scalar(x,y))

True

__4.__ Какие из нижеперечисленных векторов образуют ортонормированный базис в линейном пространстве $\mathbb{R}^{3}$:<br>
а) $(1,0,0),(0,0,1);$ <br>
б) $(1/\sqrt{2},-1/\sqrt{2},0),(1/\sqrt{2},1/\sqrt{2},0), (0,0,1);$<br>
в) $(1/2, -1/2, 0), (0, 1/2, 1/2), (0,0,1);$<br>
г) $(1,0,0),(0,1,0),(0,0,1)?$ 

In [527]:
def base_check(dim, vectors):
    result = False
    vectors_norm = []
    vector_angles = []
    if len(vectors) == dim:
        vectors_norm = [get_vector_norm(vector) for vector in vectors]
        for i, vector in enumerate(vectors):
            for new_vector in vectors[i+1:]:
                vector_angles.append(get_vector_angle(vector, new_vector))
        if np.all((np.asarray(vectors_norm)) <= 1 & (np.asarray(vectors_norm) >= 0.99)) and np.all(abs(np.asarray(vector_angles))==90):
            result = True
    return result, vectors_norm, vector_angles
    

In [528]:
base_0 = [(1,0,0),(0,0,1)]
base_1 = [(1/np.sqrt(2), -1/np.sqrt(2),0), (1/np.sqrt(2), 1/np.sqrt(2),0), (0,0,1)]
base_2 = [(1/2, -1/2, 0), (0,1/2,1/2), (0,0,1)]
base_3 = [(1,0,0), (0,1,0), (0,0,1)]

bases = [base_0, base_1, base_2, base_3]

for base in bases:
    print(base_check(dim=3, vectors=base))

(False, [], [])
(True, [0.9999999999999999, 0.9999999999999999, 1.0], [90.0, 90.0, 90.0])
(False, [0.7071067811865476, 0.7071067811865476, 1.0], [119.99999999999999, 90.0, 45.00000000000001])
(True, [1.0, 1.0, 1.0], [90.0, 90.0, 90.0])


б,г